Primo approccio per estrarre gli HOG di una immagine usando OpenCV

In [1]:
import cv2
import numpy as np

In [2]:
image = cv2.imread('Images/000040.jpg')
image.shape

(413, 550, 3)

Applico un resize all'immagine in maniera tale che sia piccolina

In [3]:
# image = cv2.resize(image, None, fx=0.3, fy=0.3)
# image = cv2.resize(image, width=min(400, image.shape[1]))
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/gabriele/.local/lib/python3.10/site-packages/cv2/qt/plugins"


In [5]:
# inizializzo il HOG person detector
winSize = (64,128)
# blockSize = (16,16)
# blockStride = (8,8)
# cellSize = (8,8)
# nbins = 9
# derivAperture = 1
# winSigma = 4.
# histogramNormType = 0
# L2HysThreshold = 2.0000000000000001e-01
# gammaCorrection = 0
# nlevels = 64
hog = cv2.HOGDescriptor(winSize)
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'HOGDescriptor'
> Overload resolution failed:
>  - HOGDescriptor() missing required argument '_blockSize' (pos 2)
>  - Can't convert object to 'str' for 'filename'


https://medium.com/@dnemutlu/hog-feature-descriptor-263313c3b40d

In [6]:
# ora faccio detecting delle persone
# Detect people in the image
locations, confidence = hog.detectMultiScale(image, padding=(4,4), scale=1.2)

In [7]:
print(locations.shape)

(4, 4)


In [8]:
locations

array([[925, 373,  80, 160],
       [167, 359, 115, 230],
       [635, 338, 134, 267],
       [290, 360, 122, 243]], dtype=int32)

In [9]:
confidence

array([2.11320335, 2.06735   , 0.59617003, 0.90654029])

In [10]:
new_image = image.copy()

In [11]:
for(x,y,w,h) in locations:
    cv2.rectangle(new_image, (x,y), (x+w, y+h), (0,0,255),2)

In [12]:
cv2.imshow('Image', new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Adesso per ogni area trovata estraggo una sottoimmagine

In [13]:
pedestrians = []

In [14]:

for (x,y,w,h) in locations:
    sub_image = image[y:y+h, x:x+w].copy()
    print(sub_image.shape)
    pedestrians.append(sub_image)

(160, 80, 3)
(230, 115, 3)
(267, 134, 3)
(243, 122, 3)


In [15]:
len(pedestrians)

4

In [24]:
for pedestrian in pedestrians:
    cv2.imshow('Pedone', pedestrian)
    cv2.waitKey(0)

cv2.destroyAllWindows()

In [25]:
pedone1, pedone2, pedone3, pedone4 = pedestrians

cv2.imshow('Pedone1', pedone1)
cv2.imshow('Pedone2', pedone2)
cv2.imshow('Pedone3', pedone3)
cv2.imshow('Pedone4', pedone4)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
pedoni = [pedone1, pedone2, pedone3, pedone4]

Per ogni pedone applico prima un resize e poi filtro gaussiano:
(w,h) = (128, 64)

In [20]:
# res = cv2.resize(img, dsize=(54, 140), interpolation=cv2.INTER_CUBIC)
# for pedone in pedoni:
#     pedone = cv2.resize(pedone, dsize=(128,64), interpolation=cv2.INTER_CUBIC)
#     print(pedone.shape)


for i in range(len(pedoni)):
    # prima faccio blurring
    pedoni[i] = cv2.GaussianBlur()
    pedoni[i] = cv2.resize(pedoni[i], dsize=(64, 128), interpolation=cv2.INTER_CUBIC)
    print(pedoni[i].shape)

(128, 64, 3)
(128, 64, 3)
(128, 64, 3)
(128, 64, 3)


In [22]:
pedone1, pedone2, pedone3, pedone4 = pedoni

cv2.imshow('Pedone1', pedone1)
cv2.imshow('Pedone2', pedone2)
cv2.imshow('Pedone3', pedone3)
cv2.imshow('Pedone4', pedone4)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [75]:
pedoni[0].shape

(160, 80, 3)

In [52]:
pedone1, pedone2, pedone3, pedone4 = pedoni

cv2.imshow('Pedone1', pedone1)
cv2.imshow('Pedone2', pedone2)
cv2.imshow('Pedone3', pedone3)
cv2.imshow('Pedone4', pedone4)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
# se volessi applicare un descrittore locale per ogni sottoimmagine
# h_pedone1 = hog.compute(pedone1)
# h_pedone1
h_pedoni = []
for pedone in pedoni:
    h = hog.compute(pedone)
    print(h.shape)
    h_pedoni.append(h)
    

(56700,)
(343980,)
(612360,)
(453600,)


https://stackoverflow.com/questions/28390614/opencv-hogdescripter-python

In [ ]:
import cv2
image = cv2.imread("test.jpg",0)
winSize = (64,128)
# blockSize = (16,16)
# blockStride = (8,8)
# cellSize = (8,8)
# nbins = 9
# derivAperture = 1
# winSigma = 4.
# histogramNormType = 0
# L2HysThreshold = 2.0000000000000001e-01
# gammaCorrection = 0
# nlevels = 64
hog = cv2.HOGDescriptor(winSize)
#compute(img[, winStride[, padding[, locations]]]) -> descriptors
winStride = (8,8)
padding = (8,8)
locations = ((10,20),)
hist = hog.compute(image,winStride,padding,locations)

Here is a snippet of code to initialize an cv2.HOGDescriptor with different parameters (The terms I used here are standard terms which are well defined in OpenCV documentation here)

Here: https://docs.opencv.org/2.4/modules/gpu/doc/object_detection.html

The resultant hog descriptor will have dimension as: 9 orientations X (4 corner blocks that get 1 normalization + 6x4 blocks on the edges that get 2 normalizations + 6x6 blocks that get 4 normalizations) = 1764. as I have given only one location for hog.compute().

In [103]:
import cv2
help(cv2.HOGDescriptor())

Help on HOGDescriptor in module cv2 object:

class HOGDescriptor(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  checkDetectorSize(...)
 |      checkDetectorSize() -> retval
 |      .   @brief Checks if detector size equal to descriptor size.
 |  
 |  compute(...)
 |      compute(img[, winStride[, padding[, locations]]]) -> descriptors
 |      .   @brief Computes HOG descriptors of given image.
 |      .       @param img Matrix of the type CV_8U containing an image where HOG features will be calculated.
 |      .       @param descriptors Matrix of the type CV_32F
 |      .       @param winStride Window stride. It must be a multiple of block stride.
 |      .       @param padding Padding
 |      .       @param locations Vector of Point
 |  
 |  computeGradient(...)
 |      computeGradient(img, grad, angleOfs[, paddin

4. Different way to initialize HOGDescriptor:
One more way to initialize is from xml file which contains all parameter values:

hog = cv2.HOGDescriptor("hog.xml")

To get an xml file one can do following:

hog = cv2.HOGDescriptor()
hog.save("hog.xml")

and edit the respective parameter values in xml file.